<a href="https://colab.research.google.com/github/salvadorhm/sqlalchemy/blob/main/sqlalchemy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SQLAlchemy Inntroducción

## Cargar las librerias de sqlachemy

In [1]:
from sqlalchemy import create_engine # Conexion con las bases de datos
from sqlalchemy import MetaData # Metadatos de las tablas
from sqlalchemy import Table, Column, Integer, String # Tipos de datos
from sqlalchemy import insert, select, update, delete # Funciones 

# Conexión con la base de datos

In [2]:
engine = create_engine('sqlite:///foo.db')

# Objeto para definir las tablas

[Definición de estructura de las tablas](https://docs.sqlalchemy.org/en/14/core/metadata.html)

In [3]:
metadata_obj = MetaData()

# Definicion de la tabla clientes

Tipos de datos de SQLAlchemy [Tipos de datos](https://www.oreilly.com/library/view/essential-sqlalchemy/9780596516147/ch04.html)

|Class name|Python type|SQL type (for SQLite driver)|Arguments|
|---|---|---|---|
|String	|string	|TEXT or VARCHAR	|length (default is unbounded)|
|Integer|	int|	INTEGER|	none|
|SmallInteger|	int	|SMALLINT|	none|
|Numeric|	float, Decimal|	NUMERIC	|precision=10 , length=2|
|Float(Numeric)|	float|	NUMERIC	|precision=10|
|DateTime|	datetime.datetime	|TIMESTAMP	|none|
|Date|	datetime.date	|DATE	|none|
|Time|	datetime.time	|TIME	|none|
|Binary|	byte string|	BLOB	|length (default is unbounded)|
|Boolean|	bool	|BOOLEAN	|none|
|Unicode|	unicode	|TEXT or VARCHAR	|length (default is unbounded)|
|PickleType	|any object that can be pickled|	BLOB	|none|
|FLOAT(Numeric)|	float, Decimal	|NUMERIC	|precision=10 ,length=2|
|TEXT(String)	|string	|TEXT	|length (default is unbounded)|
|DECIMAL(Numeric)|	float, Decimal	|NUMERIC	|precision=10,length=2|
|INT, INTEGER(Integer)|	int	|INTEGER	|none|
|TIMESTAMP(DateTime)	|datetime.datetime	|TIMESTAMP	|none|
|DATETIME(DateTime)	|datetime.datetime	|TIMESTAMP	|none|
|CLOB(String)	|string	|TEXT	|length (default is unbounded)|
|VARCHAR(String)|	string|	VARCHAR or TEXT	|length (default is unbounded)|
|CHAR(String)|	string	|CHAR or TEXT	|length (default is unbounded)|
|NCHAR(Unicode)|	string	|VARCHAR, NCHAR, or TEXT	|length (default is unbounded)|
|BLOB(Binary)|	byte string	|BLOB	|length (default is unbounded)|
|BOOLEAN(Boolean)|	bool	|BOOLEAN	|none|

In [4]:
clientes = Table(
    'clientes',
    metadata_obj,
    Column('id_cliente', Integer, primary_key=True),
    Column('nombre', String(16), nullable=False),
    Column('email', String(60), nullable=False),
)

# Creación de las tablas y de la base de datos si no existiera

In [5]:
metadata_obj.create_all(engine)

# Sentencia (statement) para insertar un nuevo registro usando SQLAlchemy Core

[Documentación para insertar](https://docs.sqlalchemy.org/en/14/tutorial/data_insert.html)

In [25]:
stmt = insert(clientes).values(nombre='Cliente 1', email="cliente1@email.com")
print(type(stmt))
print(stmt)

<class 'sqlalchemy.sql.dml.Insert'>
INSERT INTO clientes (nombre, email) VALUES (:nombre, :email)


# Conexión con la base de datos y ejecucion de la sentencia (statement).

**Nota:** with permite abrir y cerrar la sesion con la base de datos de forma automatica.



In [7]:
with engine.connect() as conn:
    result = conn.execute(stmt)

# Insertar multiple registros

In [8]:
data = [
    {"nombre":"Cliente 2","email":"cliente2@email.com"},
    {"nombre":"Cliente 3","email":"cliente3@email.com"}
    ]
stmt = insert(clientes).values(data)
print(stmt)

INSERT INTO clientes (nombre, email) VALUES (:nombre_m0, :email_m0), (:nombre_m1, :email_m1)


# Conexión con la base de datos para insertar los valores

In [9]:
with engine.connect() as conn:
    result = conn.execute(stmt)

# Sentencia (statement) para consultar todos los registros con SQLAlchemy Core

[Documentación para selección de registros](https://docs.sqlalchemy.org/en/14/orm/queryguide.html )

In [26]:
stmt = select(clientes)
print(type(stmt))
print(stmt)

<class 'sqlalchemy.sql.selectable.Select'>
SELECT clientes.id_cliente, clientes.nombre, clientes.email 
FROM clientes


# Conexión con la base de datos y ejecucion de la Sentencia (statement).

In [11]:
with engine.connect() as conn:
    result = conn.execute(stmt)
    for row in result.fetchall():
        print(f"{row['id_cliente']} : {row['nombre']} : {row['email']}")

1 : Cliente 1 : cliente1@email.com
2 : Cliente 2 : cliente2@email.com
3 : Cliente 3 : cliente3@email.com


# Sentencia (statement) para consultar un solo registro

In [27]:
stmt = select(clientes).where(clientes.c.nombre =='Cliente 1')
print(type(stmt))
print(stmt)

<class 'sqlalchemy.sql.selectable.Select'>
SELECT clientes.id_cliente, clientes.nombre, clientes.email 
FROM clientes 
WHERE clientes.nombre = :nombre_1


# Ejecutando la sentencia (statement) SELECT WHERE

In [13]:
with engine.connect() as conn:
    result = conn.execute(stmt)
    for row in result.fetchall():
        print(f"{row['id_cliente']} : {row['nombre']} : {row['email']}")

1 : Cliente 1 : cliente1@email.com


In [14]:
with engine.connect() as conn:
    result = conn.execute(stmt)
    for row in result:
        print(f"{row.id_cliente}:{row.nombre}:{row.email}")

1:Cliente 1:cliente1@email.com


## Ordenar registros

In [29]:
stmt = select(clientes).order_by(clientes.c.nombre)
print(type(stmt))
print(stmt)

<class 'sqlalchemy.sql.selectable.Select'>
SELECT clientes.id_cliente, clientes.nombre, clientes.email 
FROM clientes ORDER BY clientes.nombre


# Ejecutando la sentencia (statement) SELECT ORDER BY

In [30]:
with engine.connect() as conn:
    result = conn.execute(stmt)
    for row in result:
        print(f"{row.id_cliente}:{row.nombre}:{row.email}")

3:Cliente 3:cliente3@email.com
1:Cliente actualizado:cliente1@email.com


# Seleccionado campos a mostrar de una consulta

In [31]:
stmt = select(clientes.c.nombre).order_by(clientes.c.nombre)
print(type(stmt))
print(stmt)

<class 'sqlalchemy.sql.selectable.Select'>
SELECT clientes.nombre 
FROM clientes ORDER BY clientes.nombre


In [33]:
with engine.connect() as conn:
    result = conn.execute(stmt)
    for row in result:
        print(f"{row}")

('Cliente 3',)
('Cliente actualizado',)


# Actualizar un registro

In [15]:
stmt = (
    update(clientes).where(clientes.c.id_cliente == 1).
    values(nombre = 'Cliente actualizado')
    )

print(stmt)

UPDATE clientes SET nombre=:nombre WHERE clientes.id_cliente = :id_cliente_1


# Ejecutar la sentencia (statement) UPDATE

In [16]:
with engine.connect() as conn:
    result = conn.execute(stmt)

# Creación de un metodo para imprimir todos los registros

In [17]:
def get_all():
    with engine.connect() as conn:
        stmt = select(clientes)
        result = conn.execute(stmt)
        for row in result.fetchall():
            print(f"{row['id_cliente']} : {row['nombre']} : {row['email']}")

# Invocar el método para mostrar todos los registros

In [18]:
get_all()

1 : Cliente actualizado : cliente1@email.com
2 : Cliente 2 : cliente2@email.com
3 : Cliente 3 : cliente3@email.com


# Sentencia (statement)*texto en cursiva* para eliminar un registro con DELETE

In [19]:
stmt = delete(clientes).where(clientes.c.id_cliente == 2)
print(stmt)

DELETE FROM clientes WHERE clientes.id_cliente = :id_cliente_1


# Definir metodo para ejecutar Sentencias (statement)

In [20]:
def execute(stmt):
    with engine.connect() as conn:
        result = conn.execute(stmt)
        return result

# Llamar al metodo execute con la sentencia (statement) DELETE

In [21]:
execute(stmt)

# Imprimir todos los registros para verificar que se elimino un registro

In [22]:
get_all()

1 : Cliente actualizado : cliente1@email.com
3 : Cliente 3 : cliente3@email.com
